<a href="https://colab.research.google.com/github/edgarreis/Pen-Recognition/blob/main/Reconhecimento_de_canetas_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Arquivos necessários:
# /content/keras_model.h5     # Modelo da IA importado do https://teachablemachine.withgoogle.com/train/image
# /content/labels.txt         # Etiquetas com as classificações


from keras.models import load_model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import sys

In [ ]:
##############################################################################
# Este programa redireciona a saída padrão para um arquivo enquanto           #
# carrega um modelo de IA entreinada                                          #
# cria uma função JavaScript para transmitir vídeo de uma webcam              #
# Em seguida, define funções para converter dados de imagem JavaScript        #
# em uma imagem OpenCV e para enviar um comando JavaScript para o navegador   #
# lidar com a transmissão de vídeo.                                           #
##############################################################################

# Redirects standard output to a file
original_stdout = sys.stdout
sys.stdout = open('out.txt', 'w')

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()
class_names = [class_name.rstrip() for class_name in class_names]  # Remove whitespace and newline characters


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    var shutdown = false;  //add ~shutdown

    // Working with DOM HTML (Document Object Model)
    async function createDom() {
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      // LABEL TO INNER
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      // Video to div
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);
    }

    // Working with the frames
    async function stream_frame(label, imgData) {
      if (div === null) {
        await createDom();
      }

      var preCreate = Date.now();
      var videoRect = video.getClientRects()[0];
      captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
      var img = captureCanvas.toDataURL('image/jpeg', 0.8);
      var result = "";

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (!imgData == "") {
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });

      return {'create': preShow - preCreate,
              'capture': Date.now() - preCapture,
              'img': img};
    }

    // handle the streaming of video from a webcam
    async function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8);
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    ''')

  display(js)

def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# Sends a JavaScript command to the browser to call the stream_frame function
# The result is assigned to the data variable
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  # Returns the data received from the browser
  return data


# start streaming video from webcam
video_stream()

# label for video
label_html = 'Capturing...'
bbox = ''
teste = 0

while True:

    # Create each video frame
    js_reply = video_frame(label_html, bbox)

    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # Check if the image is not empty
    if img is not None:
        # Resize the raw image into (224-height,224-width) pixels
        image = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)

        # Make the image a numpy array and reshape it to the models input shape.
        image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

        # Normalize the image array
        image = (image / 127.5) - 1

        # Predicts the model
        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Print prediction and confidence score
        class_name_part = str(class_name[2:])
        confidence_score_part = str(np.round(confidence_score * 100))[:-2] + "%"

        # Combine the parts into a single line
        label_html = f' Capturando... Objeto: {class_name_part} - Precisão: {confidence_score_part}'

    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)

    # 27 is the ASCII for the esc key on your keyboard.
    if keyboard_input == 27:
        break

# Restaura a saída padrão para a original
sys.stdout = original_stdout

MessageError: Error: Cell has no view